In [14]:
cd /home/jovyan/GA_DSI/Projects/capstone

/home/jovyan/GA_DSI/Projects/capstone


In [15]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import lib.general_utilities as gu

%matplotlib inline

## Lemmatize data without emoji encoding.

In [ ]:
filename = 'data/processed_dataset_df/airline_proc_dataset_70percCL_noEmEnc_df'
air_df = gu.read_pickle_obj(filename)

In [6]:
air_df.head(11)

,airline,airline_sentiment,text,clean_text,stopw_clean_text,stem_stopw_clean_text,negativereason,airline_sentiment_confidence
0,Virgin America,neutral,@VirginAmerica What @dhepburn said.,what said,said,said,NaN,1.0
1,Virgin America,negative,@VirginAmerica it's really aggressive to blast...,its really aggressive to blast obnoxious enter...,really aggressive blast obnoxious entertainmen...,realli aggress blast obnoxi entertain guest fa...,Bad Flight,1.0
2,Virgin America,negative,@VirginAmerica and it's a really big bad thing...,and its a really big bad thing about it,really big bad thing,realli big bad thing,Can't Tell,1.0
3,Virgin America,negative,@VirginAmerica seriously would pay $30 a fligh...,seriously would pay 30 a flight for seats that...,seriously pay 30 seats didnt playing really ba...,serious pay 30 seat didnt play realli bad thin...,Can't Tell,1.0
4,Virgin America,positive,"@VirginAmerica it was amazing, and arrived an ...",it was amazing and arrived an hour early youre...,amazing arrived hour early youre good,amaz arriv hour earli your good,NaN,1.0
5,Virgin America,positive,@VirginAmerica I &lt;3 pretty graphics. so muc...,i lt3 pretty graphics so much better than mini...,lt3 pretty graphics better minimal iconography d,lt3 pretti graphic better minim iconographi d,NaN,1.0
6,Virgin America,positive,@VirginAmerica This is such a great deal! Alre...,this is such a great deal already thinking abo...,great deal thinking 2nd trip amp havent gone 1...,great deal think 2nd trip amp havent gone 1st ...,NaN,1.0
7,Virgin America,positive,@VirginAmerica Thanks!,thanks,thanks,thank,NaN,1.0
8,Virgin America,positive,@VirginAmerica So excited for my first cross c...,so excited for my first cross country flight l...,excited cross country lax mco ive heard great ...,excit cross countri lax mco ive heard great th...,NaN,1.0
9,Virgin America,negative,@VirginAmerica I flew from NYC to SFO last we...,i flew from nyc to sfo last week and couldnt f...,flew nyc sfo week fully sit seat large gentlem...,flew nyc sfo week fulli sit seat larg gentlema...,Bad Flight,1.0


In [17]:
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import nltk

In [18]:
# load stopwords and exclude punctuation sets.
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

In [19]:
def process_doc(text):
    
    rem_stop = " ".join([i for i in text.lower().split() if i not in stop])
    rem_punc= ''.join(ch for ch in rem_stop if ch not in exclude)
    text = " ".join(lemma.lemmatize(word) for word in rem_punc.split())
    
    return(text)

In [10]:
# Lemmatization is a more elegant way of doing this 'normalization". It finds the root of
# the word.
air_df['lemma_stopw_clean_text'] = air_df['clean_text'].apply(process_doc)

In [11]:
air_df.columns

Index(['airline', 'airline_sentiment', 'text', 'clean_text',
       'stopw_clean_text', 'stem_stopw_clean_text', 'negativereason',
       'airline_sentiment_confidence', 'lemma_stopw_clean_text'],
      dtype='object')

In [12]:
cols = ['airline', 'airline_sentiment', 'text', 'clean_text',
       'stopw_clean_text', 'stem_stopw_clean_text', 'lemma_stopw_clean_text', 'negativereason',
       'airline_sentiment_confidence']
air_df = air_df[cols]

In [13]:
air_df.head(11)

,airline,airline_sentiment,text,clean_text,stopw_clean_text,stem_stopw_clean_text,lemma_stopw_clean_text,negativereason,airline_sentiment_confidence
0,Virgin America,neutral,@VirginAmerica What @dhepburn said.,what said,said,said,said,NaN,1.0
1,Virgin America,negative,@VirginAmerica it's really aggressive to blast...,its really aggressive to blast obnoxious enter...,really aggressive blast obnoxious entertainmen...,realli aggress blast obnoxi entertain guest fa...,really aggressive blast obnoxious entertainmen...,Bad Flight,1.0
2,Virgin America,negative,@VirginAmerica and it's a really big bad thing...,and its a really big bad thing about it,really big bad thing,realli big bad thing,really big bad thing,Can't Tell,1.0
3,Virgin America,negative,@VirginAmerica seriously would pay $30 a fligh...,seriously would pay 30 a flight for seats that...,seriously pay 30 seats didnt playing really ba...,serious pay 30 seat didnt play realli bad thin...,seriously would pay 30 flight seat didnt playi...,Can't Tell,1.0
4,Virgin America,positive,"@VirginAmerica it was amazing, and arrived an ...",it was amazing and arrived an hour early youre...,amazing arrived hour early youre good,amaz arriv hour earli your good,amazing arrived hour early youre good,NaN,1.0
5,Virgin America,positive,@VirginAmerica I &lt;3 pretty graphics. so muc...,i lt3 pretty graphics so much better than mini...,lt3 pretty graphics better minimal iconography d,lt3 pretti graphic better minim iconographi d,lt3 pretty graphic much better minimal iconogr...,NaN,1.0
6,Virgin America,positive,@VirginAmerica This is such a great deal! Alre...,this is such a great deal already thinking abo...,great deal thinking 2nd trip amp havent gone 1...,great deal think 2nd trip amp havent gone 1st ...,great deal already thinking 2nd trip amp haven...,NaN,1.0
7,Virgin America,positive,@VirginAmerica Thanks!,thanks,thanks,thank,thanks,NaN,1.0
8,Virgin America,positive,@VirginAmerica So excited for my first cross c...,so excited for my first cross country flight l...,excited cross country lax mco ive heard great ...,excit cross countri lax mco ive heard great th...,excited first cross country flight lax mco ive...,NaN,1.0
9,Virgin America,negative,@VirginAmerica I flew from NYC to SFO last we...,i flew from nyc to sfo last week and couldnt f...,flew nyc sfo week fully sit seat large gentlem...,flew nyc sfo week fulli sit seat larg gentlema...,flew nyc sfo last week couldnt fully sit seat ...,Bad Flight,1.0


In [14]:
filename = 'data/processed_dataset_df/airline_proc_dataset_70percCL_noEmEnc_addLemma_df'
gu.pickle_obj(filename, air_df)

## Lemmatize data with emoji encoding.

In [20]:
filename = 'data/processed_dataset_df/airline_proc_dataset_70percCL_withEmEnc_df'
air_df = gu.read_pickle_obj(filename)

In [21]:
air_df.head(11)

,airline,airline_sentiment,text,clean_text,stopw_clean_text,stem_stopw_clean_text,emojis,negativereason,airline_sentiment_confidence
0,Virgin America,neutral,@VirginAmerica What @dhepburn said.,what said,said,said,,NaN,1.0
1,Virgin America,negative,@VirginAmerica it's really aggressive to blast...,its really aggressive to blast obnoxious enter...,really aggressive blast obnoxious entertainmen...,realli aggress blast obnoxi entertain guest fa...,,Bad Flight,1.0
2,Virgin America,negative,@VirginAmerica and it's a really big bad thing...,and its a really big bad thing about it,really big bad thing,realli big bad thing,,Can't Tell,1.0
3,Virgin America,negative,@VirginAmerica seriously would pay $30 a fligh...,seriously would pay 30 a flight for seats that...,seriously pay 30 seats didnt playing really ba...,serious pay 30 seat didnt play realli bad thin...,,Can't Tell,1.0
4,Virgin America,positive,"@VirginAmerica it was amazing, and arrived an ...",it was amazing and arrived an hour early youre...,amazing arrived hour early youre good,amaz arriv hour earli your good,,NaN,1.0
5,Virgin America,positive,@VirginAmerica I &lt;3 pretty graphics. so muc...,i lt3 pretty graphics so much better than mini...,lt3 pretty graphics better minimal iconography d,lt3 pretti graphic better minim iconographi d,,NaN,1.0
6,Virgin America,positive,@VirginAmerica This is such a great deal! Alre...,this is such a great deal already thinking abo...,great deal thinking 2nd trip havent gone 1st t...,great deal think 2nd trip havent gone 1st trip p,,NaN,1.0
7,Virgin America,positive,@VirginAmerica Thanks!,thanks,thanks,thank,,NaN,1.0
8,Virgin America,positive,@VirginAmerica So excited for my first cross c...,so excited for my first cross country flight l...,excited cross country lax mco ive heard great ...,excit cross countri lax mco ive heard great th...,,NaN,1.0
9,Virgin America,negative,@VirginAmerica I flew from NYC to SFO last we...,i flew from nyc to sfo last week and couldnt f...,flew nyc sfo week fully sit seat large gentlem...,flew nyc sfo week fulli sit seat larg gentlema...,,Bad Flight,1.0


In [22]:
air_df.columns

Index(['airline', 'airline_sentiment', 'text', 'clean_text',
       'stopw_clean_text', 'stem_stopw_clean_text', 'emojis', 'negativereason',
       'airline_sentiment_confidence'],
      dtype='object')

In [23]:
air_df['lemma_stopw_clean_text'] = air_df['clean_text'].apply(process_doc)

In [24]:
cols = ['airline', 'airline_sentiment', 'text', 'clean_text',
       'stopw_clean_text', 'stem_stopw_clean_text', 'lemma_stopw_clean_text', 
       'emojis','negativereason','airline_sentiment_confidence']
air_df = air_df[cols]

In [26]:
air_df[8:12]

,airline,airline_sentiment,text,clean_text,stopw_clean_text,stem_stopw_clean_text,lemma_stopw_clean_text,emojis,negativereason,airline_sentiment_confidence
8,Virgin America,positive,@VirginAmerica So excited for my first cross c...,so excited for my first cross country flight l...,excited cross country lax mco ive heard great ...,excit cross countri lax mco ive heard great th...,excited first cross country flight lax mco ive...,,NaN,1.0
9,Virgin America,negative,@VirginAmerica I flew from NYC to SFO last we...,i flew from nyc to sfo last week and couldnt f...,flew nyc sfo week fully sit seat large gentlem...,flew nyc sfo week fulli sit seat larg gentlema...,flew nyc sfo last week couldnt fully sit seat ...,,Bad Flight,1.0
10,Virgin America,positive,I ❤️ flying @VirginAmerica. ☺️👍,i EMOJI_1️ flying EMOJI_2️ EMOJI_3,EMOJI_1️ flying EMOJI_2️ EMOJI_3,emoji_1️ fli emoji_2️ emoji_3,emoji1️ flying emoji2️ emoji3,❤ ☺ 👍,NaN,1.0
11,Virgin America,positive,@VirginAmerica you know what would be amazingl...,you know what would be amazingly awesome bosfl...,know amazingly awesome bosfll want fly,know amazingli awesom bosfll want fli,know would amazingly awesome bosfll please wan...,,NaN,1.0


In [27]:
filename = 'data/processed_dataset_df/airline_proc_dataset_70percCL_withEmEnc_addLemma_df'
gu.pickle_obj(filename, air_df)